### Web Scrapping database with mechanicalsoup and SQlite
Here, in this project I have scrapped this [page](https://en.wikipedia.org/wiki/Comparison_of_Linux_distributions). There were several tables but I decided to save only the main table to my database.

In [1]:
# Install the package
# pip install MechanicalSoup==1.1.0

In [2]:
# Import libraries
import mechanicalsoup
import pandas as pd
import sqlite3

In [3]:
# Create browser object & open URL
browser_linux = mechanicalsoup.StatefulBrowser()
browser_linux.open("https://en.wikipedia.org/wiki/Comparison_of_Linux_distributions")

<Response [200]>

In [4]:
#Extract the table headers
table_header=browser_linux.page.find_all('th', attrs={'class': 'table-rh'})

# Get all the contents of distribution column
distribution=[value.text.replace('\n', '') for value in table_header] #replace \n with empty string
print(distribution)

['AlmaLinux', 'Alpine Linux', 'ALT Linux', 'antiX', 'ArchBang', 'Arch Linux', 'BLAG', 'Bodhi Linux', 'Canaima', 'CentOS', 'Chakra', 'Chrome OS', 'Clear Linux OS', 'ClearOS', 'CrunchBang Linux', 'Damn Small Linux', 'Debian', 'Debian Edu', 'Devuan', 'Deepin', 'Dragora GNU/Linux-Libre', 'dyne:bolic', 'Elementary OS', 'ELinOS', 'Emdebian Grip', 'EndeavourOS', 'Fedora', 'Freespire', 'Gentoo Linux', 'Guix System', 'gNewSense', 'Grml', 'Hyperbola GNU/Linux-libre', 'Kali Linux', 'Knoppix', 'Kodibuntu', 'Korora', 'LibreCMC', 'Linspire', 'Linux Mint', 'Linux Lite', 'Mageia', 'Mandriva Linux', 'Manjaro Linux', 'MEPIS', 'Musix GNU+Linux', 'Netrunner', 'NixOS', 'Novell Open Enterprise Server', 'OpenELEC', 'openSUSE', 'OpenWrt', 'OpenMandriva Lx', 'Oracle Linux', 'Parabola GNU/Linux-libre', 'Pardus', 'Parrot OS', 'Parsix', 'Parted Magic', 'PCLinuxOS', 'Pop! OS', 'Pentoo', 'Porteus', 'Puppy Linux', 'PureOS', 'Qubes OS', 'Red Hat Enterprise Linux (RHEL)', 'Red Hat Linux', 'ROSA', 'Rocks Cluster Distri

In [5]:
# This is giving all the contents of Distribution column from all of the tables.
# Let's specify the index till which table we want to save it in our database
# I want to save the first table and the last content of distribution column of first table is Zorin OS. Let's check its index.
print(distribution.index('Zorin OS'))


95


In [6]:
# So index is 95. now lets slice it
distribution=distribution[:96]
print(distribution)

['AlmaLinux', 'Alpine Linux', 'ALT Linux', 'antiX', 'ArchBang', 'Arch Linux', 'BLAG', 'Bodhi Linux', 'Canaima', 'CentOS', 'Chakra', 'Chrome OS', 'Clear Linux OS', 'ClearOS', 'CrunchBang Linux', 'Damn Small Linux', 'Debian', 'Debian Edu', 'Devuan', 'Deepin', 'Dragora GNU/Linux-Libre', 'dyne:bolic', 'Elementary OS', 'ELinOS', 'Emdebian Grip', 'EndeavourOS', 'Fedora', 'Freespire', 'Gentoo Linux', 'Guix System', 'gNewSense', 'Grml', 'Hyperbola GNU/Linux-libre', 'Kali Linux', 'Knoppix', 'Kodibuntu', 'Korora', 'LibreCMC', 'Linspire', 'Linux Mint', 'Linux Lite', 'Mageia', 'Mandriva Linux', 'Manjaro Linux', 'MEPIS', 'Musix GNU+Linux', 'Netrunner', 'NixOS', 'Novell Open Enterprise Server', 'OpenELEC', 'openSUSE', 'OpenWrt', 'OpenMandriva Lx', 'Oracle Linux', 'Parabola GNU/Linux-libre', 'Pardus', 'Parrot OS', 'Parsix', 'Parted Magic', 'PCLinuxOS', 'Pop! OS', 'Pentoo', 'Porteus', 'Puppy Linux', 'PureOS', 'Qubes OS', 'Red Hat Enterprise Linux (RHEL)', 'Red Hat Linux', 'ROSA', 'Rocks Cluster Distri

In [7]:
# Now extract the table data
table_data=browser_linux.page.find_all('td')
columns=[value.text.replace('\n', '') for value in table_data]
#print(columns)
# only take those data which we want to save. let's check we are taking right data or not.
print(columns.index('AlmaLinux Foundation'))
print(columns.index('Binary blobs'))


6
1062


In [8]:
# Let's slice the column accordin to those index value which information we need
columns=columns[6:1062]
#print(columns)

In [9]:
# Let's save all the columns information. In the main table we have 11 column
# organizing columns will be like
#columns[0:][::11]
#columns[1:][::11]
column_names = ["Founder", 
                "Maintainer", 
                "Initial_Release_Year", 
                "Current_Stable_Version", 
                "Security_Updates", 
                "Release_Date", 
                "System_Distribution_Commitment", 
                "Forked_From", 
                "Target_Audience", 
                "Cost", 
                "Status"]
# make a dictionary
dictionary={'Distribution': distribution} #key is dstribution and value it will take from the previous save distribution output
for idx, key in enumerate(column_names):
    dictionary[key]=columns[idx:][::11]
    

# Create dataframe
linux_df=pd.DataFrame(data=dictionary) 

#print(df.head())
#print(df.tail())

In [10]:
#Insert data into a database.Let's make a connection to the database
connection = sqlite3.connect("comparison_of_linux_distribution.db")

# Create an object to execute the connection
cursor = connection.cursor()

# create database table and insert all data frame rows
cursor.execute("create table linux (Distribution, " + ",".join(column_names)+ ")")
for i in range(len(linux_df)):
    cursor.execute("insert into linux values (?,?,?,?,?,?,?,?,?,?,?,?)", linux_df.iloc[i]) # '?' for the elements

# Permanently save Data to the Comparison of Linux distribution
connection.commit()

connection.close()

In [11]:
# We also can save data as csv file
linux_df.to_csv('linux_comparison.csv', index=False)